In [1]:
import spacy
from tqdm.notebook import tqdm
import pandas as pd

In [6]:
nlp = spacy.load("fr_core_news_sm")
brute_sentences_df = pd.read_csv('../data/dataset/brute_sentences_2.csv')
cities_df = pd.read_csv('../data/cities.csv', sep=',')

In [ ]:
cities_df["label"].sample(40)

In [16]:
def apply_iob_to_sentence(doc_sentence, clean_df, departure, arrival):
    for word in [token_word.text for token_word in doc_sentence]:
        if (word in ["departure", "arrival", "none", "intermediary"]):
            plus_sub_words = cities_df.sample()['label'].values[0].lower()
            plus_sub_words = plus_sub_words.split()
            sub_words = (departure.split() if word == "departure" else arrival.split())

            for sub_word_idx, sub_word in enumerate((sub_words if word in ["departure", "arrival"] else plus_sub_words)):
                if (sub_word_idx == 0) :
                    clean_df.append(
                        {
                            "word": sub_word,
                            "status": "B"
                        }
                    )
                    continue
                clean_df.append(
                    {
                        "word": sub_word,
                        "status": "I"
                    }
                )
            continue
        clean_df.append(
            {
                "word": word,
                "status": "O"
            }
        )

In [17]:
clean_df = [{"word": "END", "status": "O"}]

with tqdm(total= 30 * len(brute_sentences_df), unit="sentence") as pbar:
    for idx, sentence in enumerate(brute_sentences_df["sentence"]):
        sentence = sentence.lower()
        for i in range(30):
            departure = cities_df.sample()["label"].values[0].lower()
            arrival = cities_df.sample()["label"].values[0].lower()

            while(arrival == departure):
                arrival = cities_df.sample()["label"].values[0].lower()

            doc_sentence = nlp(sentence)

            apply_iob_to_sentence(
                doc_sentence,
                clean_df,
                departure,
                arrival
            )

            clean_df.append(
                {
                    "word": "END",
                    "status": "O" 
                }               
            )
            pbar.update(1)

clean_df = pd.DataFrame(clean_df)

  0%|          | 0/4560 [00:00<?, ?sentence/s]

In [20]:
new_df = pd.DataFrame(columns=["sub_sentence", "status"])

with tqdm(
    total=clean_df.shape[0] - 8,
    unit="sentence") as pbar:
    for idx, row in clean_df.iloc[:-8].iterrows():
        sub_sentence = f"{clean_df.iloc[idx - 1].word}[{clean_df.iloc[idx - 1].status}] "
        additional_words = " ".join([word for word in clean_df.iloc[idx : idx + 7].word])
        sub_sentence += additional_words

        new_row = {
            "sub_sentence": sub_sentence,
            "status": clean_df.iloc[idx].status
        }
        new_df = pd.concat([new_df, pd.DataFrame([new_row])], ignore_index=True)
        pbar.update(1)

  0%|          | 0/106027 [00:00<?, ?sentence/s]

In [21]:
new_df_shuffled = new_df.sample(frac=1)
train_df = new_df_shuffled[:int(len(new_df_shuffled)*0.7)]
valid_df = new_df_shuffled[int(len(new_df_shuffled)*0.7):int(len(new_df_shuffled)*0.85)]
test_df = new_df_shuffled[int(len(new_df_shuffled)*0.85):]

In [22]:
train_df.to_csv('../data/dataset/ner_train.csv', index=False)
valid_df.to_csv('../data/dataset/ner_valid.csv', index=False)
test_df.to_csv('../data/dataset/ner_test.csv', index=False)